In [1]:
import numpy as np
import torch, pdb, csv
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.autograd import Variable
from itertools import ifilter
from random import randint
from joblib import Parallel, delayed
from torch.optim import lr_scheduler
from random import shuffle

In [2]:
BATCH_SIZE = 130000
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, hid_dim, pretrained=None):
        super(Word2Vec, self).__init__()
        self.hid_dim = hid_dim
        
        #these are by intent to learn separate embedding matrices, we return word_emb
        self.word_emb = nn.Embedding(vocab_size, hid_dim)
        if pretrained is not None:
            self.word_emb.weight.data.copy_(pretrained)
        
        self.context_emb = nn.Embedding(vocab_size, hid_dim)
        if pretrained is not None:
            self.context_emb.weight.data.copy_(pretrained)        
        
        self.sigmoid = nn.LogSigmoid()
        
    def forward(self, wrd, cntxt, labels):
        wrd_vec = self.word_emb(wrd) # N * 1 * D
        cntxt_vec = self.context_emb(cntxt) # N * 5 * D
        res = torch.bmm(wrd_vec, cntxt_vec.view(BATCH_SIZE, self.hid_dim, -1))
        res = res.squeeze(1)
        res = res * labels
        res = self.sigmoid(res)
        
        # these are N * (1 + neg_exmpl) logsigmoid values
        # for each mini-batch we have a probability score for the 5 contexts
        # return res
        
        return (torch.sum(res)*-1.0)/res.size(0)

In [7]:
def get_sim(wrd, k, mat, word2index):
    if wrd not in word2index:
        return None
    vec = mat[word2index[wrd], :].unsqueeze(1)
    othrs = torch.mm(mat, vec)
    othrs, ind = torch.sort(othrs, 0, descending=True)
    topk = ind[:k]
    for i in range(topk.size()[0]):
        print(index2word[topk[i][0]])

def get_score(wrd1, wrd2, mat, word2index):
    return torch.dot(mat[word2index[wrd1],:], mat[word2index[wrd2], :])

def get_glovedict(glove_path):
    vocab_d = set()
    with open(glove_path) as f:
        for line in f:
            word, vec = line.split(' ', 1)
            word = word.strip().lower()
            vocab_d.add(word)
            
    return vocab_d
    
def get_gloveready(glove_path, vocab_size, dim, word2index):
    pretrained_weight = torch.FloatTensor(vocab_size, dim)
    fnd = 0
    with open(glove_path) as f:
        for line in f:
            word, vec = line.split(' ', 1)
            word = word.strip().lower()
            if word in word2index:
                ind = word2index[word]
                pretrained_weight[ind, :] = torch.from_numpy(np.array(list(map(float, vec.split()))))
                fnd += 1

    print('Found {0} words with glove vectors, total was {1}'.format(fnd, vocab_size))
    return pretrained_weight

def process_lines(data):
    pairs, vocab = set(), {}
    for cn, l in enumerate(data):
        dt = l.split("|||")
        score = float(dt[3].split(" ")[1].split("=")[1])
        if score < 3.3:
            continue
        wrd1, wrd2 = dt[1], dt[2]
        wrd1, wrd2 = wrd1.strip(), wrd2.strip()

        if ".pdf" not in wrd1 and ".pdf" not in wrd2 and wrd1.isalpha() and wrd2.isalpha():
            sc = editdist_score(wrd1, wrd2)
            if sc > min(len(wrd1), len(wrd2))/2 + 2:
                if wrd1 + " " + wrd2 not in pairs and wrd2 + " " + wrd1 not in pairs:
                    pairs.add(wrd1 + " " + wrd2)
                    if wrd1 not in vocab:
                        vocab[wrd1] = 1
                    else:
                        vocab[wrd1] += 1

                    if wrd2 not in vocab:
                        vocab[wrd2] = 1
                    else:
                        vocab[wrd2] += 1

    return pairs, vocab

def get_vocab(min_freq, flName=None, lines=None):
    if flName is not None:
        with open(flName) as fp:
            lines = fp.readlines()
    
    return process_lines(lines)

def get_chunks(lines, cn):
    chunks = []
    chunk_size = len(lines)//cn
    for i in range(0, chunk_size*cn + 1):
        chunk = lines[i*chunk_size:i*chunk_size + chunk_size]
        chunks.append(chunk)
    return chunks

def editdist_score(p1, p2):
    n, m = len(p1), len(p2)
    dp = [[0 for x in range(m+1)] for x in range(n+1)]

    for i in range(n+1):
        for j in range(m+1):
            if i == 0:
                dp[0][j] = j
            elif j == 0:
                dp[i][0] = i            
            elif p1[i-1] == p2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1]) + 1
    return dp[n][m]

def filter_data(pairs, word2index):
    new_pairs = set()
    fp = open("ppdb-processed.txt", "w")
    for line in pairs:        
        p1, p2 = line.split(" ")
        if p1 in word2index and p2 in word2index:
            new_pairs.add(p1 + " " + p2)
            fp.write(line)
            fp.write("\n")
            
    fp.close()
    return new_pairs

In [30]:
def load_opposites():
    oppos = {}
    with open('sneha_antonyms.csv', 'r') as csvfile:
        next(csvfile)
        data = csv.reader(csvfile, delimiter=',')
        for row in data:
            if row[1] == 'antonym' and float(row[2]) > 0.8:
                wrd1, wrd2 = row[3], row[4]
                wrd1 = wrd1.strip().lower()
                wrd2 = wrd2.strip().lower()
                
                if wrd1 not in oppos:
                    oppos[wrd1] = []
                oppos[wrd1].append(wrd2)
                if wrd2 not in oppos:
                    oppos[wrd2] = []
                oppos[wrd2].append(wrd1)
    return oppos

def init_train(glove_path, dim, min_count, neg_exmpl):
    g_vocab = get_glovedict(glove_path)
    pairs, tok_freq = get_vocab(min_count, flName="ppdb-2.0-l-lexical")
    opposites = load_opposites()
    for wrd in opposites:
        if wrd not in tok_freq:
            tok_freq[wrd] = 0
        tok_freq[wrd] += 1
        
        antos = opposites[wrd]
        for awrd in antos:
            if awrd not in tok_freq:
                tok_freq[awrd] = 0
            tok_freq[awrd] += 1
            
    vocab = set(tok_freq.keys())
    vocab = vocab.intersection(g_vocab)

    word2index, index2word = {}, {}
    
    for wrd in vocab:
        if tok_freq[wrd] >= min_count:
            index2word[len(index2word)] = wrd
            word2index[wrd] = len(index2word) - 1
        else:
            tok_freq[wrd] = 0
            
    pairs = filter_data(pairs, word2index)
    print("Data ready: {} {} {}".format(len(index2word), len(pairs), len(vocab)))
    return pairs, word2index, index2word, vocab, tok_freq, opposites


In [31]:
glove_path, dim, min_count, neg_exmpl = "glove.6B.50d.txt", 50, 1, 10
pairs, word2index, index2word, vocab, tok_freq, opposites = init_train(glove_path, dim, min_count, neg_exmpl)    

Data ready: 17585 103416 17585
Data ready: 17585 103416 17585


In [78]:
mdl = Word2Vec(len(index2word), dim)    
mdl.load_state_dict(torch.load('./mdl_skipgme6.pth', map_location=lambda storage, loc: storage))
w2vmat = mdl.word_emb.weight.data.cpu()
wnorm = torch.norm(w2vmat, 2, 1, True)

While copying the parameter named word_emb.weight, whose dimensions in the model are torch.Size([17585, 50]) and whose dimensions in the checkpoint are torch.Size([21530, 50]), ...


RuntimeError: inconsistent tensor size, expected tensor [17585 x 50] and src [21530 x 50] to have the same number of elements, but got 879250 and 1076500 elements respectively at /opt/conda/conda-bld/pytorch_1502006348621/work/torch/lib/TH/generic/THTensorCopy.c:86

In [77]:
get_sim('occur', 10, w2vmat/wnorm, word2index)

occur
occurring
affect
arise
localized
due
likely
result
possibly
occurrences


In [52]:
get_sim('occur', 10, w2vmat, word2index)

occur
occurring
arise
affect
possibly
due
normal
localized
outbreaks
possible


In [27]:
get_sim('eat', 10, w2vmat, word2index)

eat
shepherd
slumped
unshackled
craftspeople
detonation
weighed
presentations
deception
electrical


In [28]:
get_sim('fear', 10, w2vmat, word2index)

fear
guidance
propuestas
fora
centralized
narrates
pressured
replace
attracting
disassociated


In [11]:
get_sim('hi', 10, w2vmat, word2index)

hi
ho
ai
ok
wow
techs
oh
wanna
yo
tu


In [ ]:
get_sim('salary', 10, w2vmat, word2index)

In [29]:
get_sim('coast', 10, w2vmat, word2index)

coast
tetroxide
fate
bettering
collecting
indisputable
reside
grenades
happy
recorders


In [14]:
get_score('coast', 'shore', w2vmat, word2index)

0.8319517970085144

In [30]:
get_score('young', 'new', w2vmat, word2index)

0.7724562287330627

In [31]:
get_score('east', 'west', w2vmat, word2index)

-0.05693214759230614

In [17]:
get_score('take', 'obtain', w2vmat, word2index)

0.8381562829017639

In [18]:
get_score('receive', 'give', w2vmat, word2index)

0.886364758014679

In [19]:
get_score('shrinks', 'small', w2vmat, word2index)

0.7134382724761963

In [20]:
get_sim('shrink', 10, w2vmat, word2index)

shrink
shrinking
shrinks
shrank
shrunk
increase
slowing
reduction
reduce
dramatically


In [21]:
get_sim('monster', 10, w2vmat, word2index)

monster
monsters
kid
movie
hell
killer
scary
bugs
gigantic
crazy


In [22]:
get_score('shrinks', 'grow', w2vmat, word2index)

0.5169721245765686